In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 784)               3136      
_________________________________________________________________
dense_3 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_4 (Batch (None, 300)               1200      
_________________________________________________________________
dense_4 (Dense)              (None, 300)               90300     
_________________________________________________________________
batch_normalization_5 (Batch (None, 300)               1200      
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [3]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [5]:
model.layers[1].updates

C:\Users\jh\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1331: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


[]

In [11]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
tf.random.set_seed(42)
np.random.seed(42)

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [12]:
X_train_A.shape

(43986, 28, 28)

In [13]:
X_train_B.shape

(200, 28, 28)

In [14]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [16]:
tf.random.set_seed(42)
np.random.seed(42)

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

C:\Users\jh\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.9770 - accuracy: 0.8132 - val_loss: 0.4202 - val_accuracy: 0.8548
Epoch 2/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.3408 - accuracy: 0.8801 - val_loss: 0.3191 - val_accuracy: 0.8904
Epoch 3/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2987 - accuracy: 0.8931 - val_loss: 0.3224 - val_accuracy: 0.8929
Epoch 4/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2753 - accuracy: 0.9022 - val_loss: 0.2938 - val_accuracy: 0.9031
Epoch 5/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2579 - accuracy: 0.9092 - val_loss: 0.2741 - val_accuracy: 0.9121
Epoch 6/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2453 - accuracy: 0.9132 - val_loss: 0.2737 - val_accuracy: 0.9068
Epoch 7/20
1375/1375 [==============================] - 2s 1ms/step - loss: 0.2332 - accuracy: 0.9162 - val_loss: 0.2776 - val_accuracy:

In [17]:
model_A.save("my_model_A.h5")

In [18]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [19]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [20]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

C:\Users\jh\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [21]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 27ms/step - loss: 0.2374 - accuracy: 0.9050 - val_loss: 0.2869 - val_accuracy: 0.8895
Epoch 2/4
7/7 [==============================] - 0s 7ms/step - loss: 0.2011 - accuracy: 0.9400 - val_loss: 0.2623 - val_accuracy: 0.9006
Epoch 3/4
7/7 [==============================] - 0s 5ms/step - loss: 0.1825 - accuracy: 0.9550 - val_loss: 0.2439 - val_accuracy: 0.9138
Epoch 4/4
7/7 [==============================] - 0s 5ms/step - loss: 0.1680 - accuracy: 0.9600 - val_loss: 0.2291 - val_accuracy: 0.9178


In [22]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 1s 28ms/step - loss: 0.1074 - accuracy: 0.9800 - val_loss: 0.1213 - val_accuracy: 0.9523
Epoch 2/16
7/7 [==============================] - 0s 5ms/step - loss: 0.0408 - accuracy: 0.9950 - val_loss: 0.0901 - val_accuracy: 0.9655
Epoch 3/16
7/7 [==============================] - 0s 4ms/step - loss: 0.0268 - accuracy: 1.0000 - val_loss: 0.0764 - val_accuracy: 0.9736
Epoch 4/16
7/7 [==============================] - 0s 5ms/step - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.0701 - val_accuracy: 0.9746
Epoch 5/16
7/7 [==============================] - 0s 5ms/step - loss: 0.0168 - accuracy: 1.0000 - val_loss: 0.0658 - val_accuracy: 0.9767
Epoch 6/16
7/7 [==============================] - 0s 5ms/step - loss: 0.0143 - accuracy: 1.0000 - val_loss: 0.0626 - val_accuracy: 0.9767
Epoch 7/16
7/7 [==============================] - 0s 5ms/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.0619 - val_accuracy: 0.9777
Epoch 8/16
7/7 [=================